
# Reconnaissance faciale via un arbre de décision de décision binaire

## Partie 2: Reconnaissance faciale en temps réel

## Librairies

In [1]:
import pickle
import numpy as np

import cv2

from sklearn import tree # < = importation du package d'algo binary tree via la librairie scikit-learn

## Algorithme

In [2]:
# Variable vers le dataset
DATASET = "../dataset/data/"

with open(DATASET+'/visages.pkl', 'rb') as fh:
    visages = pickle.load(fh)

with open(DATASET+'/noms.pkl', 'rb') as fh:
    noms = pickle.load(fh)

print('Shape of visages matrix --> ', visages.shape)

Shape of visages matrix -->  (20, 50, 50, 3)


# Arbre binaire de décision (Decision Tree) :

Les arbres binaires de décisions sont des  modèles d'apprentissages automatique qui utilisent des structures en forme d'arbres pour prendre des décisions. Le but est de trouver le meilleur découpage des données du dataset. Dans un arbre de déscision chaque nœud de l'arbre représente une caractéristique (ou un attribut), chaque branche représente une décision basée sur cette caractéristique, et chaque feuille représente la sortie (la classe ou la valeur prédite).

In [3]:
"""
Ce code entraîne un classificateur d'arbre de décision en utilisant l'ensemble de données donné de visages et de noms correspondants.

params:
- visages : tableau numpy représentant les images de visage, remodelé en tableau 2D avec forme (N, M), où N est le nombre d’échantillons et M le nombre de caractéristiques.
- noms : liste des noms correspondant aux images des visages.

returns:
- clf : classificateur d'arbre de décision formé.
"""

N = len(noms)

visages = visages.reshape(N, -1)

clf = tree.DecisionTreeClassifier()
clf.fit(visages, noms)


DecisionTreeClassifier()

## Exécution

In [4]:
# On importe coordonées des visages
cascade_visage = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

camera = cv2.VideoCapture(0) # 0 pour 'built-in' caméra, 1 pour caméra externe

while True:
    
    ret, trame = camera.read()
    if ret == True:
        
        gris = cv2.cvtColor(trame, cv2.COLOR_BGR2GRAY)
        coordonnees_visage = cascade_visage.detectMultiScale(gris, 1.3, 5)

        for (x, y, l, h) in coordonnees_visage:
            
            visage = trame[y:y + h, x:x + l, :]
            visage_redimensionne = cv2.resize(visage, (50, 50)).flatten().reshape(1,-1)
            
            texte = clf.predict(visage_redimensionne)
            
            cv2.putText(trame, texte[0], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
            cv2.rectangle(trame, (x, y), (x + l, y + l), (0, 0, 255), 2)

        cv2.imshow('Reconnaissance faciale en temps réel', trame)
        
        if cv2.waitKey(1) == 27:
            break
            
    else:
        
        print("erreur")
        break

cv2.destroyAllWindows()
camera.release()